In [1]:
# 이미지 인식 - CNN
## MNIST 사례

In [2]:
import numpy  as np
import tensorflow as tf
from tensorflow import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.callbacks import ModelCheckpoint, EarlyStopping

import os
import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [3]:
# seed 값 설정
seed = 0
np.random.seed(seed)
tf.random.set_seed(seed)

In [11]:
# 데이터 불러오기
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(-1, 28, 28, 1).astype('float32') / 255
x_test = x_test.reshape(-1, 28, 28, 1).astype('float32') / 255
y_train = tf.keras.utils.to_categorical(y_train)
y_test = tf.keras.utils.to_categorical(y_test)

In [12]:
x_train.shape

(60000, 28, 28, 1)

In [13]:
y_train.shape

(60000, 10)

In [20]:
# 컨볼루션 신경망 설정

model = Sequential([Conv2D(32, kernel_size=(1,1), input_shape=(28,28,1), activation='relu'),
                   Conv2D(64, (3, 3), activation='relu'), MaxPooling2D(pool_size=2),
                   Dropout(0.25), Flatten(), Dense(128, activation='relu'),
                   Dropout(0.5), Dense(10, activation='softmax')])

model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_13 (Conv2D)           (None, 28, 28, 32)        64        
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 26, 26, 64)        18496     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 13, 13, 64)        0         
_________________________________________________________________
dropout_13 (Dropout)         (None, 13, 13, 64)        0         
_________________________________________________________________
flatten_7 (Flatten)          (None, 10816)             0         
_________________________________________________________________
dense_13 (Dense)             (None, 128)               1384576   
_________________________________________________________________
dropout_14 (Dropout)         (None, 128)              

In [6]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [7]:
# 모델 최적화 설정
MODEL_DIR = './model1/'
if not os.path.exists(MODEL_DIR) :
    os.mkdir(MODEL_DIR)

In [8]:
modelpath = MODEL_DIR + "mnist-cnn-{epoch:02d}-{val_loss:.4f}.hdf5"
checkpointer = ModelCheckpoint(filepath=modelpath, monitor='val_loss',
                              verbose=1, save_best_only=True)
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=10)

In [9]:
# 모델의 실행
history = model.fit(x_train, y_train, validation_data=(x_test, y_test), 
                    epochs=30, batch_size=200,
                   callbacks=[early_stopping_callback, checkpointer])

Train on 60000 samples, validate on 10000 samples
Epoch 1/30
60000/60000 [==============================] - 44s 735us/step - loss: 0.2872 - accuracy: 0.9120 - val_loss: 0.0545 - val_accuracy: 0.9826

Epoch 00001: val_loss improved from inf to 0.05450, saving model to ./model1/mnist-cnn-01-0.0545.hdf5
Epoch 2/30
60000/60000 [==============================] - 43s 723us/step - loss: 0.0893 - accuracy: 0.9739 - val_loss: 0.0424 - val_accuracy: 0.9865

Epoch 00002: val_loss improved from 0.05450 to 0.04245, saving model to ./model1/mnist-cnn-02-0.0424.hdf5
Epoch 3/30
60000/60000 [==============================] - 44s 726us/step - loss: 0.0700 - accuracy: 0.9793 - val_loss: 0.0344 - val_accuracy: 0.9880

Epoch 00003: val_loss improved from 0.04245 to 0.03444, saving model to ./model1/mnist-cnn-03-0.0344.hdf5
Epoch 4/30
60000/60000 [==============================] - 44s 730us/step - loss: 0.0546 - accuracy: 0.9834 - val_loss: 0.0403 - val_accuracy: 0.9873

Epoch 00004: val_loss did not improv

In [ ]:
from keras.models import load_model
del model1
model1 = load_model('model1/mnist-cnn-09-0.0295.hdf5')

In [ ]:
# 테스트 정확도 출력
print("\n Test Accuracy : %.4f" % (model1.evaluate(x_test, y_test, verbose=0)[1]))

In [ ]:
# 테스트셋의 오차
y_vloss = history.history['val_loss']

# 학습셋의 오차
y_loss = history.history['loss']

In [ ]:
# 그래프로 표현
x_len = np.arange(1, len(y_loss)+1)
plt.figure(figsize=(8,6))
plt.plot(x_len, y_vloss, marker='.', c='red', label='Testset_loss')
plt.plot(x_len, y_loss, marker='.', c='blue', label='Trainset_loss')

# 그래프에 그리드를 주고 레이블을 표시
plt.legend(loc='upper right')
# plt.axie([0, 20, 0, 0.35])
plt.grid()
plt.xlabel('epoch')
plt.ylabel('loss')
plt.show()

In [ ]:
# 모델의 실행
history = model.fit(x_train, y_train, validation_data=(x_test, y_test), 
                    epochs=50, batch_size=20,
                   callbacks=[early_stopping_callback, checkpointer])

In [ ]:
# 테스트 정확도 출력
print("\n Test Accuracy : %.4f" % (model.evaluate(x_test, y_test, verbose=0)[1]))